# Before you start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [11]:
# import numpy and pandas
import numpy as np
import pandas as pd


# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents. Please, load the data using Ironhack's database (db: employees, table: employees_advanced).

In [12]:
# Your code here:
df = pd.read_csv('data/hypothesis_testing_dataset.csv')

num_rows, num_columns = df.shape
print(f'El dataframe tiene {num_rows} filas y {num_columns} columnas')

El dataframe tiene 33183 filas y 8 columnas


Examine the `salaries` dataset using the `head` function below.

In [13]:
# Your code here:
df.head()


Name                              Job Titles  \
0    AARON,  JEFFERY M                                SERGEANT   
1      AARON,  KARINA   POLICE OFFICER (ASSIGNED AS DETECTIVE)   
2  AARON,  KIMBERLEI R                CHIEF CONTRACT EXPEDITER   
3  ABAD JR,  VICENTE M                       CIVIL ENGINEER IV   
4    ABASCAL,  REECE E             TRAFFIC CONTROL AIDE-HOURLY   

         Department Full or Part-Time Salary or Hourly  Typical Hours  \
0            POLICE                 F           Salary            NaN   
1            POLICE                 F           Salary            NaN   
2  GENERAL SERVICES                 F           Salary            NaN   
3       WATER MGMNT                 F           Salary            NaN   
4              OEMC                 P           Hourly           20.0   

   Annual Salary  Hourly Rate  
0       101442.0          NaN  
1        94122.0          NaN  
2       101592.0          NaN  
3       110064.0          NaN  
4            NaN        19.86

We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [14]:
# Your code here:
df.isna().sum()


Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [15]:
# Your code here:
filter_full_time = df["Salary or Hourly"] == 'Salary'
filter_part_time = df["Salary or Hourly"] == 'Hourly'

num_full_time_employees = df[filter_full_time].shape[0]
num_part_time_employees = df[filter_part_time].shape[0]

print(f"There are {num_rows} employees: {num_full_time_employees} salaried and {num_part_time_employees} hourly")


There are 33183 employees: 25161 salaried and 8022 hourly


What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [16]:
# Your code here:
df["Department"].value_counts()


POLICE                   13414
FIRE                      4641
STREETS & SAN             2198
OEMC                      2102
WATER MGMNT               1879
AVIATION                  1629
TRANSPORTN                1140
PUBLIC LIBRARY            1015
GENERAL SERVICES           980
FAMILY & SUPPORT           615
FINANCE                    560
HEALTH                     488
CITY COUNCIL               411
LAW                        407
BUILDINGS                  269
COMMUNITY DEVELOPMENT      207
BUSINESS AFFAIRS           171
COPA                       116
BOARD OF ELECTION          107
DoIT                        99
PROCUREMENT                 92
INSPECTOR GEN               87
MAYOR'S OFFICE              85
CITY CLERK                  84
ANIMAL CONTRL               81
HUMAN RESOURCES             79
CULTURAL AFFAIRS            65
BUDGET & MGMT               46
ADMIN HEARNG                39
DISABILITIES                28
TREASURER                   22
HUMAN RELATIONS             16
BOARD OF

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [17]:
# Your code here:
from scipy.stats import ttest_1samp

salary_to_compare = 30
significance = 0.05 # 95%

p_value = ttest_1samp(df[filter_part_time]["Hourly Rate"], salary_to_compare).pvalue

text_h0 = 'H0 -> no podemos rechazar H0: el salario es de 30$ por hora'
text_h1 = 'H1 -> rechazamos H0: el salario es significativamente diferente de 30$ por hora'
print(text_h1 if p_value < significance else text_h0)

H1 -> rechazamos H0: el salario es significativamente diferente de 30$ por hora


We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [45]:
# Your code here:
salary_to_compare = 86000.0
significance = 0.05 # 95%

filter_police_workers = df["Department"] == "POLICE"

# Hay valores nulos en el Annual Salary del departamento POLICE.
print(f"Valores nulos: {df[filter_police_workers]['Annual Salary'].isna().sum()}")

# Creo un filtro nuevo para quitar esos valores nulos
filter_salary_not_nan = df["Annual Salary"].notna()
police_department_salaries = df[filter_police_workers & filter_salary_not_nan]["Annual Salary"].to_list()

t_statistic, p_value = ttest_1samp(police_department_salaries, salary_to_compare)

# Como el test es one-sided divido p_value por 2
p_value = p_value / 2

# Definimos las hipótesis:
# - H0: los salarios son iguales o menores que el año anterior
# - H1: los salarios son mayores que el año pasado
if t_statistic > 0 and p_value < significance:
    print("Rechazamos H0: los salarios son mayores que el año pasado")
else:
    print("No podmeos rechazar H0. Los salarios no son mayores que el año pasado")


Valores nulos: 10
Rechazamos H0: los salarios son mayores que el año pasado


Using the `crosstab` function, find the department that has the most hourly workers. 

In [46]:
# Your code here:
department = pd.crosstab(df["Department"], df["Salary or Hourly"]).sort_values(by="Hourly", ascending=False).iloc[0].name

print(f"El departamente con mas hourly workers es: {department}")

El departamente con mas hourly workers es: STREETS & SAN


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [56]:
# Your code here:
salary_to_compare = 35
significance = 0.05 # 95%

filter_department = df["Department"] == "STREETS & SAN"

# Hay valores nulos
print(f"Valores nulos: {df[filter_department]['Hourly Rate'].isna().sum()}")

# Hago un filtro para ignorar esos valores
filter_salary_not_nan = df["Hourly Rate"].notna()
department_hourly_rate = df[filter_department & filter_salary_not_nan]["Hourly Rate"].to_list()

t_statistic, p_value = ttest_1samp(department_hourly_rate, salary_to_compare)

# Como el test es one-sided divido p_value por 2
p_value = p_value / 2

# Definimos las hipótesis:
# - H0: los salarios son iguales o mayores que el año anterior
# - H1: los salarios son menores que el año pasado
if t_statistic < 0 and p_value < significance:
    print("Rechazamos H0: los salarios son menores que el año pasado")
else:
    print("No podemos rechazar H0. Los salarios no son menores que el año pasado")


Valores nulos: 336
Rechazamos H0: los salarios son menores que el año pasado


# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [74]:
# Your code here:
import scipy

filter_hourly_workers = df["Salary or Hourly"] == "Hourly"
filter_salary_not_nan = df["Hourly Rate"].notna()

hourly_workers_salaries = df[filter_hourly_workers & filter_salary_not_nan]["Hourly Rate"]

confidence_level = 0.95
salaries_mean = hourly_workers_salaries.mean()
degrees_of_freedom = len(hourly_workers_salaries) - 1
salaries_standard_error = scipy.stats.sem(hourly_workers_salaries)

confidence_min, confidence_max =scipy.stats.t.interval(confidence_level, degrees_of_freedom, salaries_mean, salaries_standard_error)

print(f"El intervalo de confianza esta entre {confidence_min} y {confidence_max}")

El intervalo de confianza esta entre 32.52345834488425 y 33.05365708767623


Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [75]:
# Your code here:
filter_department = df["Department"] == "POLICE"
filter_salary_not_nan = df["Annual Salary"].notna()

salaries = df[filter_department & filter_salary_not_nan]["Annual Salary"]

confidence_level = 0.95
salaries_mean = salaries.mean()
degrees_of_freedom = len(salaries) - 1
salaries_standard_error = scipy.stats.sem(salaries)

confidence_min, confidence_max = scipy.stats.t.interval(confidence_level, degrees_of_freedom, salaries_mean, salaries_standard_error)

print(f"El intervalo de confianza esta entre {confidence_min} y {confidence_max}")

El intervalo de confianza esta entre 86177.05631531784 y 86795.77269094894


# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [13]:
# Your code here:

